In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import gc

/Applications/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('Loading data ...')

Loading data ...


In [5]:
train = pd.read_csv('./train_2016_v2.csv')
prop = pd.read_csv('./properties_2016.csv')
sample = pd.read_csv('./sample_submission.csv')

/Applications/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90275 entries, 0 to 90274
Data columns (total 3 columns):
parcelid           90275 non-null int64
logerror           90275 non-null float64
transactiondate    90275 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 2.1+ MB


In [7]:
prop.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [10]:
prop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
parcelid                        int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottub

In [12]:
sample.head(10)

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0
5,10898347,0,0,0,0,0,0
6,10933547,0,0,0,0,0,0
7,10940747,0,0,0,0,0,0
8,10954547,0,0,0,0,0,0
9,10976347,0,0,0,0,0,0


In [13]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 7 columns):
ParcelId    int64
201610      int64
201611      int64
201612      int64
201710      int64
201711      int64
201712      int64
dtypes: int64(7)
memory usage: 159.4 MB


In [14]:
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

In [27]:
prop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
parcelid                        int64
airconditioningtypeid           float32
architecturalstyletypeid        float32
basementsqft                    float32
bathroomcnt                     float32
bedroomcnt                      float32
buildingclasstypeid             float32
buildingqualitytypeid           float32
calculatedbathnbr               float32
decktypeid                      float32
finishedfloor1squarefeet        float32
calculatedfinishedsquarefeet    float32
finishedsquarefeet12            float32
finishedsquarefeet13            float32
finishedsquarefeet15            float32
finishedsquarefeet50            float32
finishedsquarefeet6             float32
fips                            float32
fireplacecnt                    float32
fullbathcnt                     float32
garagecarcnt                    float32
garagetotalsqft                 float32
hashottub

In [15]:
df_train = train.merge(prop, how = 'left', on = 'parcelid')

In [28]:
df_train.head()

NameError: name 'df_train' is not defined

In [16]:
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis = 1)
y_train = df_train['logerror'].values

In [17]:
print(x_train.shape, y_train.shape)

(90275, 55) (90275,)


In [18]:
train_columns = x_train.columns

In [19]:
for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

In [20]:
del df_train; gc.collect()

587

In [21]:
split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

In [22]:
d_train = xgb.DMatrix(x_train, label = y_train)
d_valid = xgb.DMatrix(x_valid, label = y_valid)

In [23]:
del x_train, x_valid; gc.collect()

14

In [29]:
print(y_train)

[ 0.0276 -0.1684 -0.004  ...,  0.2127 -0.0294 -0.0305]


In [31]:
print(d_train)

In [32]:
params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metrics'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

In [33]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds = 100, verbose_eval = 10)

[0]	train-rmse:0.506042	valid-rmse:0.497867
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[10]	train-rmse:0.423742	valid-rmse:0.415403
[20]	train-rmse:0.358395	valid-rmse:0.349918
[30]	train-rmse:0.307095	valid-rmse:0.298423
[40]	train-rmse:0.267407	valid-rmse:0.258636
[50]	train-rmse:0.237214	valid-rmse:0.228452
[60]	train-rmse:0.214682	valid-rmse:0.206015
[70]	train-rmse:0.198188	valid-rmse:0.189617
[80]	train-rmse:0.186315	valid-rmse:0.177839
[90]	train-rmse:0.177872	valid-rmse:0.16957
[100]	train-rmse:0.171952	valid-rmse:0.163871
[110]	train-rmse:0.167818	valid-rmse:0.159943
[120]	train-rmse:0.164952	valid-rmse:0.157143
[130]	train-rmse:0.162955	valid-rmse:0.155319
[140]	train-rmse:0.161566	valid-rmse:0.154111
[150]	train-rmse:0.160561	valid-rmse:0.153184
[160]	train-rmse:0.159852	valid-rmse:0.152619
[170]	train-rmse:0.159362	valid-rmse:0.152299
[180]	train-rmse:0.158954	valid-rmse:0

In [35]:
print(watchlist)

[(<xgboost.core.DMatrix object at 0x14a3442e8>, 'train'), (<xgboost.core.DMatrix object at 0x14a344908>, 'valid')]


In [36]:
del d_train, d_valid

In [37]:
sample['parcelid'] =sample['ParcelId']
df_test = sample.merge(prop, on = 'parcelid', how = 'left')

In [39]:
print(gc.collect())

378


In [40]:
df_test.head()

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid,airconditioningtypeid,architecturalstyletypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0,0,0,0,0,0,10754147,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,0,0,0,0,0,0,10759547,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,0,0,0,0,0,0,10843547,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.369141,NaN,NaN,NaN
3,10859147,0,0,0,0,0,0,10859147,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.570312,NaN,NaN,NaN
4,10879947,0,0,0,0,0,0,10879947,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.169922,NaN,NaN,NaN


In [41]:
sample.head()

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid
0,10754147,0,0,0,0,0,0,10754147
1,10759547,0,0,0,0,0,0,10759547
2,10843547,0,0,0,0,0,0,10843547
3,10859147,0,0,0,0,0,0,10859147
4,10879947,0,0,0,0,0,0,10879947


In [42]:
del prop; gc.collect()

779

In [44]:
x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [45]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2985217 entries, 0 to 2985216
Data columns (total 55 columns):
airconditioningtypeid           float32
architecturalstyletypeid        float32
basementsqft                    float32
bathroomcnt                     float32
bedroomcnt                      float32
buildingclasstypeid             float32
buildingqualitytypeid           float32
calculatedbathnbr               float32
decktypeid                      float32
finishedfloor1squarefeet        float32
calculatedfinishedsquarefeet    float32
finishedsquarefeet12            float32
finishedsquarefeet13            float32
finishedsquarefeet15            float32
finishedsquarefeet50            float32
finishedsquarefeet6             float32
fips                            float32
fireplacecnt                    float32
fullbathcnt                     float32
garagecarcnt                    float32
garagetotalsqft                 float32
hashottuborspa                  bool
heatingors

In [46]:
del df_test, sample; gc.collect()

58

In [49]:
d_test = xgb.DMatrix(x_test)

In [50]:
del x_test; gc.collect()

149

In [51]:
p_test = clf.predict(d_test)

In [52]:
del d_test; gc.collect()

0

In [53]:
sub = pd.read_csv('./sample_submission.csv')

In [54]:
for c in sub.columns[sub.columns != 'ParceId']:
    sub[c] = p_test

In [55]:
sub.to_csv('xgb_starter.csv', index = False, float_format = '%.4f')